# Julia - Day 2

https://adventofcode.com/2021/day/2

## Data

In [11]:
using CSV
using DataFrames

input = CSV.read("Input/2_1.txt", DataFrame, delim = " ", header = false)
rename!(input, [:instruction, :n_step])
example = DataFrame(
    instruction = String["forward", "down", "forward", "up", "down", "forward"],
    n_step = Int64[5, 5, 8, 3, 8, 2]
)

,instruction,n_step
,String,Int64
1,forward,5
2,down,5
3,forward,8
4,up,3
5,down,8
6,forward,2


In [20]:
first(input, 5)

,instruction,n_step
,String7,Int64
1,forward,9
2,down,3
3,down,8
4,forward,2
5,up,3


## Part 1

### Function

In [ ]:
function drive!(instructions)
    #is instruction for depth or horizontal axis
    transform!(
        instructions,
        :instruction => ByRow(x -> x in ["up", "down"] ? "depth" : "horizontal") => :instruction_type
    )

    #positive or negative step
    transform!(
        instructions,        
        [:instruction, :n_step]=> ByRow((x, y) -> x in ["forward", "down"] ? y : -y) => :step
    )

    #compute position at each step
    transform!(
        instructions,        
        [:instruction_type, :step] => ByRow((x, y) -> x == "depth" ? y : 0) => :depth,
        [:instruction_type, :step] => ByRow((x, y) -> x == "horizontal" ? y : 0) => :horizontal
    )

    transform!(
        instructions,        
        :depth => cumsum => :depth,
        :horizontal => cumsum => :horizontal
    )

end

drive! (generic function with 1 method)

### Result

In [106]:
#Example
drive!(example)
result = example[end, :depth] * example[end, :horizontal]
println(
    string("Example : ", result)
)

#Input
drive!(input)
result = input[end, :depth] * input[end, :horizontal]
println(
    string("Input : ", result)
)


Example : 150


Input : 1451208


## Part 2

### Function

In [7]:
function drive!(instructions)
    #is instruction for depth or horizontal axis
    transform!(
        instructions,
        :instruction => ByRow(x -> x in ["up", "down"] ? "aim" : "horizontal") => :instruction_type
    )

    #positive or negative step
    transform!(
        instructions,        
        [:instruction, :n_step]=> ByRow((x, y) -> x in ["forward", "down"] ? y : -y) => :step
    )

    #compute position at each step
    ## Add 0 when not changing aim or horizontal
    transform!(
        instructions,        
        [:instruction_type, :step] => ByRow((x, y) -> x == "aim" ? y : 0) => :aim,
        [:instruction_type, :step] => ByRow((x, y) -> x == "horizontal" ? y : 0) => :horizontal
    )

    ## Cumulative aim at each step

    transform!(
        instructions,        
        :aim => cumsum => :aim
    )

    ## Depth increase at each step
    transform!(
        instructions,
        [:aim, :horizontal] => ByRow((x, y) ->  x * y) => :depth
    )

    ## Cumulative depth and horizontal at each step
    transform!(
        instructions,
        :horizontal => cumsum => :horizontal,
        :depth => cumsum => :depth
    )

end

drive! (generic function with 1 method)

### Result

In [8]:
#Example
drive!(example)
result = example[end, :depth] * example[end, :horizontal]
println(
    string("Example : ", result)
)

#Input
drive!(input)
result = input[end, :depth] * input[end, :horizontal]
println(
    string("Input : ", result)
)


Example : 900
Input : 1620141160


In [9]:
example

,instruction,n_step,instruction_type,step,aim,horizontal,depth
,String,Int64,String,Int64,Int64,Int64,Int64
1,forward,5,horizontal,5,0,5,0
2,down,5,aim,5,5,5,0
3,forward,8,horizontal,8,5,13,40
4,up,3,aim,-3,2,13,40
5,down,8,aim,8,10,13,40
6,forward,2,horizontal,2,10,15,60
